In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [4]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [5]:
train_images.shape

(60000, 28, 28)

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [8]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [9]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [10]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [11]:
modelpath = MODEL_DIR + "mnist-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                              verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [12]:
history = model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=30,
                    batch_size=200, #epoch=5, verbose=2,
                    callbacks=[early_stopping_callback, checkpointer])

Train on 60000 samples, validate on 10000 samples
Epoch 1/30


InvalidArgumentError:  Received a label value of 9 which is outside the valid range of [0, 4).  Label values: 9 9 3 6 1 6 1 8 8 1 2 1 6 9 5 1 6 5 3 9 1 6 8 3 0 7 2 9 5 9 5 6 2 9 9 5 3 1 9 5 4 3 8 2 4 6 4 3 3 9 6 8 4 9 6 9 9 9 3 5 0 9 9 0 6 6 7 9 1 2 6 3 7 8 1 5 6 8 4 3 1 0 0 1 3 5 6 2 6 1 6 6 8 9 2 9 7 0 7 4 3 5 8 4 9 3 4 2 2 5 6 0 4 2 9 4 8 7 8 6 3 6 9 7 7 7 3 1 8 2 5 1 0 2 4 4 1 5 0 8 1 1 4 6 8 5 0 2 1 1 0 7 4 7 8 4 2 3 1 2 8 7 2 7 6 8 5 1 6 9 2 5 4 4 0 5 9 8 6 3 3 0 1 2 1 4 5 0 9 3 2 0 2 9 6 5 0 5 7 0
	 [[node loss/dense_2_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_1270]

Function call stack:
keras_scratch_graph


In [ ]:
# history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30,
#                     batch_size=200, #epoch=5, verbose=2,
#                     callbacks=[early_stopping_callback, checkpointer])